# DNABERT-2 Test

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True).to('cuda')

/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a608677312175eb6f8143d/bert_layers.py:125: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expe

In [2]:
from SeReGen.dataset_builder import DatasetBuilder, SILVA_header_parser
builder = DatasetBuilder(SILVA_header_parser)
dataset = builder.from_fasta(['./SILVA_138.1_SSURef_NR99_tax_silva.fasta'])

In [3]:
import scipy.stats as st
import numpy as np
target_zscore = st.norm.ppf(.9)
lengths = dataset['seqs'].apply(len)
mean = np.mean(lengths)
std = np.std(lengths)
max_len = int(target_zscore * std + mean)
max_len

1669

In [5]:
import importlib
import embedding_gen
embedding_gen = importlib.reload(embedding_gen)

In [4]:
from embedding_gen import Generator
import multiprocessing as mp
import pandas as pd
def create_process(seqs: pd.Series, dev: str, show_output=False):
    ctx = mp.get_context('spawn')
    parent_conn, child_conn = ctx.Pipe()
    gen = Generator(tokenizer, model.to(dev), dev, max_len)
    proc = ctx.Process(target=gen.gen_embeddings_batched, args=(seqs.tolist(), show_output, child_conn))
    proc.start()
    return proc, parent_conn

procs = [create_process(dataset['seqs'].iloc[len(dataset)//3*i:len(dataset)//3*(i+1)],
                       f'cuda:{i}', show_output=i == 0) for i in range(3)]
for job, conn in procs:
    job.join()
results = [conn.recv() for job, conn in procs]

  0%|          | 0/14180 [00:00<?, ?it/s]/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a608677312175eb6f8143d/bert_layers.py:432: UserWarning: Increasing alibi size from 512 to 1041
  warnings.warn(
  0%|          | 1/14180 [00:00<2:48:03,  1.41it/s]/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a608677312175eb6f8143d/bert_layers.py:432: UserWarning: Increasing alibi size from 1041 to 1558
  warnings.warn(
  0%|          | 10/14180 [00:05<2:07:00,  1.86it/s]/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a608677312175eb6f8143d/bert_layers.py:432: UserWarning: Increasing alibi size from 512 to 766
  warnings.warn(
  0%|          | 13/14180 [00:07<1:55:07,  2.05it/s]/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a60867731217

/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a608677312175eb6f8143d/bert_layers.py:432: UserWarning: Increasing alibi size from 1392 to 1443
  warnings.warn(
/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a608677312175eb6f8143d/bert_layers.py:432: UserWarning: Increasing alibi size from 1443 to 1669
  warnings.warn(
Process SpawnProcess-2:
Traceback (most recent call last):
  File "/opt/apps/intel19/python3/3.9.7/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/apps/intel19/python3/3.9.7/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/work/08947/nkalidas/ls6/SeqRep/embedding_gen.py", line 30, in gen_embeddings_batched
    result = [self.gen_embeddings(ser[i*12:(i+1)*12]) for i in it]
  File "/work/08947/nkalidas/ls6/SeqRep/embedding_gen.py",

EOFError: 

In [ ]:
from tqdm.notebook import tqdm
import gc
results = []
for i in tqdm(range(len(dataset))):
    inputs = tokenizer([dataset['seqs'].iloc[i]], return_tensors='pt', padding=True,
                       truncation=True, max_length=max_len).to('cuda')
    hidden_states = model(inputs['input_ids'])[0]
    results.append(torch.mean(hidden_states, dim=1).to('cpu').detach())
    del hidden_states
    del inputs
    gc.collect()
    torch.cuda.empty_cache()
results = torch.concatenate(results)
torch.cuda.memory_allocated() / 1000 / 1000 / 1000

  0%|          | 0/510508 [00:00<?, ?it/s]

/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a608677312175eb6f8143d/bert_layers.py:432: UserWarning: Increasing alibi size from 1023 to 1041
  warnings.warn(
/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a608677312175eb6f8143d/bert_layers.py:432: UserWarning: Increasing alibi size from 1041 to 1558
  warnings.warn(
/home1/08947/nkalidas/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/5fd206e1a13cee3ef4a608677312175eb6f8143d/bert_layers.py:432: UserWarning: Increasing alibi size from 1558 to 1669
  warnings.warn(
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (s

In [59]:
import pickle
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(results, f)

In [6]:
import pickle
with open('embeddings.pkl', 'rb') as f:
    results = pickle.load(f)

EOFError: Ran out of input

In [10]:
from SeReGen.pipeline import Pipeline
p = Pipeline()
p.dataset = dataset
p.preproc_reprs = dataset['seqs']
p.reprs = results.numpy()

In [11]:
p.reprs

array([[-0.09292029,  0.01425622,  0.05715077, ...,  0.24194978,
         0.397345  ,  0.07131382],
       [-0.077975  , -0.12892923,  0.11243261, ...,  0.26336786,
         0.36144704,  0.04833413],
       [-0.06565865, -0.04209089,  0.11501783, ...,  0.28261527,
         0.46087557,  0.09688981],
       ...,
       [-0.09044188, -0.01524179,  0.06210067, ...,  0.22145635,
         0.39046147,  0.06370578],
       [-0.10373005, -0.02744277,  0.08305177, ...,  0.28199852,
         0.41193148,  0.08838928],
       [-0.12806076, -0.08461487,  0.13926002, ...,  0.20326377,
         0.34901994,  0.08177941]], dtype=float32)

In [17]:
rng = np.random.default_rng()
query_ind = rng.integers(0, len(p.dataset), 1)[0]
query = p.dataset['seqs'].iloc[query_ind]
query

'UUCUGCGCGCGUNCUAUACUGCAGUGGACGCAGUUGAUUCACCGGAGCUUGCUCCAUCGAAUUAACUGAGUCGCGAACGGGUGAGUAACGCGUAGGUAACCUGUCUUUUAGCGGGGGAUAACUAUUGGAAACGAUAGCUAAUACCGCAUAAGAGUGACUAACACAUGUUAGUGAUUUAAAAGAUACAAUUGUAUCACUAAGAGAUGGACCUGCGUUGUAUUAGCUAGUUGGUAGGGUAGAGGCCUACCAAGGCGACGAUACAUAGCCGACCUGAGAGGGUGAACGGCCACACUGGGACUGAGACACGGCCCAGACUCCUACGGGAGGCAGCAGUAGGGAAUCUUCGGCAAUGGGGGGAACCCUGACCGAGCAACGCCGCGUGAGUGAAGAAGGUUUUCGGAUCGUAAAGCUCUGUUGUCAGAGACGAACGUAUGUGAGAGUGGAAAGUUCACAUAGUGACGGUAACUGACCAGAAAGGGACGGCUAACUACGUGCCAGCAGCCGCGGUAAUACGUAGGUCCCGAGCGUUGUCCGGAUUUAUUGGGCGUAAAGCGAGCGCAGGCGGUUGUAUAAGUCAGAAGUAAAAGGCAGUGGCUCAACCAUUGUGUGCUUUUGAAACUGUAUGACUUGAGUGCAGAAGGGGAGAGUGGAAUUCCAUGUGUAGCGGUGAAAUGCGUAGAUAUAUGGAGGAACACCGGUGGCGAAAGCGGCUCUCUGGUCUGUGACUGACGCUGAGGCUCGAAAGCGUGGGUAGCGAACAGGAUUAGAUACCCUGGUAGUCCACGCCGUAAACGCUGAGUGCUAGGUGUUAGGCCCUUUCCGGGGCUUAGUGCCGGAGCUAACGAUUAAGCACUCCGCCUGGGGAGUACGACCGCAAGGUUGAAACUCAAAGGAAUUGACGGGGGCCGCACAAGCGGUGGACAUGUGGUUUAAUUCGAAGCAACGCGAAGAACCUUACCAGGUCUUGACAUCCCGGUGCUAUUCUUAGAGAUAAGAAGU

In [12]:
from SeReGen.distance import Alignment
class Model:
    def __init__(self):
        self.distance = Alignment(768, sample_size=100)

    def transform(self, seqs, batch_size):
        tokens = tokenizer(list(seqs), return_tensors='pt', padding=True,
                           truncation=True, max_length=max_len).to('cuda')
        hidden_states = model(tokens['input_ids'])[0]
        result = torch.mean(hidden_states, dim=1).to('cpu').detach()
        del hidden_states
        del tokens
        return result.numpy().reshape(1, -1)
p.model = Model()

class MyDecoder:
    def transform(self, dists, *args, **kwargs):
        return dists
p.decoder = MyDecoder()

In [14]:
p.transform(p.dataset['seqs'].iloc[0], 1)

array([[0.16418135, 0.04805302, 0.16118021, ..., 0.08686286, 0.3593892 ,
        0.06751855]], dtype=float32)

In [50]:
!git pull
import time
import multiprocessing as mp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import Image
from SeReGen.pipeline import HomologousSequencePipeline, RNASequencePipeline
from SeReGen.visualize import reprs_by_label

def load(kind, path: str):
    p = kind.load(path)
    p.load_dataset(['SILVA_138.1_SSURef_NR99_tax_silva.fasta'], header_parser='SILVA')
    return p

class Visualizer:
    def __init__(self, p, axes: tuple):
        self.p = p
        self.axes = axes
        self.mask = p.dataset['labels'].apply(lambda i: len(i) == 7)
        self.reprs = p.reprs[self.mask][..., self.axes]
        self.ds = p.dataset[self.mask]
    
    def vis_domain(self):
        reprs_by_label(self.reprs, self.ds, 'Domain', 'Representations by Domain', savepath='out.png')
        plt.close()
        
    def vis_phylum(self):
        mask2 = self.p.dataset['labels'][self.mask].label_mask('Domain', 'Bacteria')
        reprs_by_label(self.reprs, self.ds, 'Phylum', 'Representations by Phylum', savepath='out.png', mask=mask2,
                       filter=2000)
        plt.close()
        
def rolling_average(arr, width):
    return np.convolve(arr, np.ones(width), 'valid') / width

class Search:
    def __init__(self, p, program: str):
        self.p = p
        self.program = program
        self.query = None
        self.results, results_names, results_tax = None, None, None
        self.dists = None
        self.program_output = None
        self.positive_hits = None
        self.program_names = None
        
    def gen_query(self):
        rng = np.random.default_rng()
        self.query = rng.choice(self.p.dataset['seqs'], 1)[0]
        with open('query.fasta', 'w') as f:
            f.write('>query\n')
            f.write(self.query)
        print(f'The query is {len(self.query)} base pairs long.')
        
    def run_program(self):
        if self.program == 'BLAST':
            !source blast-search.sh
            self.program_output = pd.read_csv('blast-results.csv', names=['query', 'name', 'pct_ident', 'score'] +
                                              [f'unk{i}' for i in range(6)] + ['e_value', 'bit_score'])
            self.program_output = self.program_output[:500]
        if self.program == 'SSEARCH':
            pairs = zip([self.query] * len(p.dataset), p.preproc_reprs)
            with mp.Pool(120) as pool:
                aln = np.fromiter(tqdm(pool.imap(p.model.distance.transform, pairs, chunksize=32),
                                       total=len(p.dataset)), dtype=np.float64)
            self.sort = np.argsort(aln)[::-1]
            sort = self.sort[:500]
            self.program_output = pd.DataFrame({
                'name': p.dataset.iloc[sort]['raw_headers'].apply(lambda i: i.split(' ')[0]),
                'score': aln[sort]
            })
            
    def my_results(self):
        start = time.time()
        self.dists, self.results = p.search(self.query, n_neighbors=500)
        print('My total search time: ', time.time() - start)

        self.dists = self.dists.flatten()
        self.results_names = np.array([i['raw_headers'].split(' ')[0] for i in self.results])
        self.results_tax = np.array([i['labels'] for i in self.results])
        
    def calc_positive_hits(self):
        self.program_names = self.program_output['name'].tolist()
        self.positive_hits = np.array([i in self.program_names for i in self.results_names])
        first_zero = list(self.positive_hits.astype(np.int64)).index(0)
        print(f'Num hits: {np.sum(self.positive_hits)}, Contiguous matches: {first_zero}')
        
    def plot_hit_rate(self):
        hit_rate = rolling_average(self.positive_hits, 25)
        plt.plot(hit_rate)
        plt.title('Average Search Hit Rate')
        plt.xlabel('Result Number')
        plt.ylabel('Fraction of NN Results Also Returned by BLAST')
        plt.ylim([0, 1])
        plt.savefig('out.png')
        plt.close()
        return hit_rate
        
    def plot_hit_err(self):
        hit_locs = np.array([[idx, self.program_names.index(val)] for idx, val in enumerate(self.results_names) \
                             if val in self.program_names])
        hit_errors = np.abs(hit_locs[:, 1] - hit_locs[:, 0])
        plt.plot(rolling_average(hit_errors, 10))
        plt.title('Search Hit Error Rate')
        plt.xlabel('Search Hit Number')
        plt.ylabel('Difference Between Positions in Both Searches')
        plt.ylim([0, 500])
        plt.savefig('out.png')
        plt.close()
        return hit_errors
        
    def pipeline(self):
        self.gen_query()
        self.run_program()
        self.my_results()
        self.calc_positive_hits()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Already up to date.


In [ ]:
s = Search(p, 'SSEARCH')
s.pipeline()
s.plot_hit_rate()
Image('out.png')

The query is 1444 base pairs long.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/510508 [00:00<?, ?it/s]

In [ ]:
s.plot_hit_err()
Image('out.png')